In [ ]:
import sys, os
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')


In [ ]:
from azureml.core import Experiment

experiment_name = "ner_new"
experiment = Experiment(ws, name=experiment_name)


In [ ]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-gpu', file_path = 'conda.yaml')

# Specify a GPU base image
pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails

src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      arguments=["--json_config", "training_config.json"],
                      compute_target="zd-ner",
                      environment=pytorch_env)
run = experiment.submit(src)
RunDetails(run).show()

In [ ]:
from azureml.train.hyperdrive import RandomParameterSampling, BanditPolicy, HyperDriveConfig, uniform, PrimaryMetricGoal

param_sampling = RandomParameterSampling( {
        'lr': uniform(0.01, 1),
        'adam_epsilon': uniform(0.01, 1)
    }
)

early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=10)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling, 
                                     policy=early_termination_policy,
                                     primary_metric_name='eval_f1',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=2,
                                     max_concurrent_runs=4)

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

In [ ]:
print(f'Best Run is:\n  eval_f1: {0:.5f} \n  learning rate: {1:.5f} \n  adam_epsilon: {2:.5f}'
.format(
        best_run_metrics['eval_f1'],
        best_run_metrics['lr'],
        best_run_metrics['adam_epsilon'])
     )

In [ ]:
best_run = azureml.core.ScriptRun(experiment=Experiment(ws, name="ner"),run_id="ner_new_1606688308_21ae6f00")

In [ ]:
model = best_run.register_model(model_name = 'ner', model_path = './outputs')
print(model.name, model.id, model.version, sep = '\t')

In [69]:
#remote deploy
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model

dockerfile = r"""
FROM mcr.microsoft.com/azureml/base:openmpi3.1.2-ubuntu18.04
RUN apt install openssl
RUN conda install python=3.6.2 numpy>=1.17.0 pandas>=1.0.0 scikit-learn=0.22.1 pip=20.1.1
RUN pip install azureml-defaults torch nltk transformers==3.0.2 fast-bert
RUN pip uninstall Werkzeug
RUN pip install Werkzeug==0.16.1
RUN python -c "import nltk; nltk.download('punkt')"
"""

env = Environment.from_conda_specification(name="env", file_path="conda.yaml")
# env = Environment.from_dockerfile(name="env", dockerfile=dockerfile)

inference_config = InferenceConfig(entry_script="score.py", environment=env)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=10, 
                                               tags={'data': 'text',  'method':'DistillBert-NER', 'framework':'pytorch'},
                                               description='Classify PER, LOC, ORG, and MISC')

service = Model.deploy(workspace=ws, 
                           name='neri', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)
service.wait_for_deployment(True)
print(service.state)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.....................................................................

KeyboardInterrupt: 

In [ ]:
print(service.get_logs())

In [ ]:
print("Scoring URI is: {}".format(service.scoring_uri))

In [ ]:
import json
import requests

response = requests.post(url=service.scoring_uri, data=json.dumps("Steve went to London"),headers={"Content-type": "application/json"})
print(response.text)

